In [1]:
import re
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import (
    StructType,
    StructField,
    StringType,
    IntegerType,
    DoubleType,
)

In [3]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder.appName("Teen_Addictions")
    .config("spark.executor.memory", "4g")
    .getOrCreate()
)

spark.sparkContext.setLogLevel("ERROR")
# Tip to reader: use WARN for development, ERROR in prod

In [4]:
df = spark.read.csv(
    path="dataset/teen_phone_addiction_dataset.csv",
    header=True,
    mode="PERMISSIVE",
)

In [5]:
df.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- School_Grade: string (nullable = true)
 |-- Daily_Usage_Hours: string (nullable = true)
 |-- Sleep_Hours: string (nullable = true)
 |-- Academic_Performance: string (nullable = true)
 |-- Social_Interactions: string (nullable = true)
 |-- Exercise_Hours: string (nullable = true)
 |-- Anxiety_Level: string (nullable = true)
 |-- Depression_Level: string (nullable = true)
 |-- Self_Esteem: string (nullable = true)
 |-- Parental_Control: string (nullable = true)
 |-- Screen_Time_Before_Bed: string (nullable = true)
 |-- Phone_Checks_Per_Day: string (nullable = true)
 |-- Apps_Used_Daily: string (nullable = true)
 |-- Time_on_Social_Media: string (nullable = true)
 |-- Time_on_Gaming: string (nullable = true)
 |-- Time_on_Education: string (nullable = true)
 |-- Phone_Usage_Purpose: string (nullable 